# Imports

In [1]:
import json
import requests
import pandas as pd
import plotly.express as px

# Data

In [2]:
latitude = [
    42.25539,
    42.25623,
    42.25701,
    42.25869,
    42.26185,
    42.26574,
    42.2692,
    42.26919,
    42.27179,
    42.2773,
    42.28136,
    42.28746,
    42.29308,
    42.29589,
    42.29616,
    42.29527,
    42.28392,
    42.27219,
    42.27121,
]

longitude = [
    -85.63683,
    -85.63883,
    -85.64067,
    -85.64743,
    -85.6479,
    -85.64834,
    -85.64828,
    -85.6513,
    -85.65437,
    -85.65581,
    -85.6568,
    -85.66019,
    -85.66268,
    -85.66178,
    -85.65403,
    -85.64806,
    -85.64832,
    -85.64818,
    -85.64851,
]


In [3]:
df = pd.DataFrame({"latitude": latitude, "longitude": longitude})


In [4]:
fig = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    color_discrete_sequence=["black"],
    mapbox_style="open-street-map",
    zoom=12,
)

fig.update_layout(margin={"l": 0, "r": 0, "t": 0, "b": 0})
fig.show()


Format route waypoints for API request

In [5]:
waypoints = {
    f"waypoint{i}": f"{coords[0]},{coords[1]}"
    for i, coords in enumerate(zip(df["latitude"], df["longitude"]))
}


In [6]:
waypoints


{'waypoint0': '42.25539,-85.63683',
 'waypoint1': '42.25623,-85.63883',
 'waypoint2': '42.25701,-85.64067',
 'waypoint3': '42.25869,-85.64743',
 'waypoint4': '42.26185,-85.6479',
 'waypoint5': '42.26574,-85.64834',
 'waypoint6': '42.2692,-85.64828',
 'waypoint7': '42.26919,-85.6513',
 'waypoint8': '42.27179,-85.65437',
 'waypoint9': '42.2773,-85.65581',
 'waypoint10': '42.28136,-85.6568',
 'waypoint11': '42.28746,-85.66019',
 'waypoint12': '42.29308,-85.66268',
 'waypoint13': '42.29589,-85.66178',
 'waypoint14': '42.29616,-85.65403',
 'waypoint15': '42.29527,-85.64806',
 'waypoint16': '42.28392,-85.64832',
 'waypoint17': '42.27219,-85.64818',
 'waypoint18': '42.27121,-85.64851'}

# Request

In [7]:
url = "https://fleet.ls.hereapi.com/2/calculateroute.json"

In [8]:
with open('API_KEY.json') as f:
    data = json.load(f)
API_KEY = data["API_KEY"]
params = {"apiKey": API_KEY}


Add route matching mode

In [9]:
params.update(
    {
        "routeMatch": 1,
        "mode": "fastest;car;traffic:disabled",
    }
)


Add GPS waypoints

In [10]:
params.update(waypoints)


Show final request parameters (with API key censored)

In [11]:
{k: (v if k != "apiKey" else "**********") for k, v in params.items()}

{'apiKey': '**********',
 'routeMatch': 1,
 'mode': 'fastest;car;traffic:disabled',
 'waypoint0': '42.25539,-85.63683',
 'waypoint1': '42.25623,-85.63883',
 'waypoint2': '42.25701,-85.64067',
 'waypoint3': '42.25869,-85.64743',
 'waypoint4': '42.26185,-85.6479',
 'waypoint5': '42.26574,-85.64834',
 'waypoint6': '42.2692,-85.64828',
 'waypoint7': '42.26919,-85.6513',
 'waypoint8': '42.27179,-85.65437',
 'waypoint9': '42.2773,-85.65581',
 'waypoint10': '42.28136,-85.6568',
 'waypoint11': '42.28746,-85.66019',
 'waypoint12': '42.29308,-85.66268',
 'waypoint13': '42.29589,-85.66178',
 'waypoint14': '42.29616,-85.65403',
 'waypoint15': '42.29527,-85.64806',
 'waypoint16': '42.28392,-85.64832',
 'waypoint17': '42.27219,-85.64818',
 'waypoint18': '42.27121,-85.64851'}

Request and response data

In [12]:
response = requests.get(url, params=params)
data = response.json()
data


{'response': {'route': [{'mode': {'type': 'matched',
     'transportModes': ['car'],
     'trafficMode': 'disabled'},
    'boatFerry': False,
    'railFerry': False,
    'waypoint': [{'linkId': '+115644923',
      'mappedPosition': {'latitude': 42.25528, 'longitude': -85.63712},
      'originalPosition': {'latitude': 42.25539, 'longitude': -85.63683},
      'spot': 0.70249,
      'confidenceValue': 0.15,
      'elevation': 0.0,
      'headingDegreeNorthClockwise': 10000.0,
      'headingMatched': -28.0,
      'matchDistance': 34.13,
      'minError': 27.0,
      'routeLinkSeqNrMatched': 0,
      'speedMps': 0.0,
      'timestamp': 0},
     {'linkId': '+1049349902',
      'mappedPosition': {'latitude': 42.25621, 'longitude': -85.63868},
      'originalPosition': {'latitude': 42.25623, 'longitude': -85.63883},
      'spot': 0.13988,
      'confidenceValue': 0.29,
      'elevation': 0.0,
      'headingDegreeNorthClockwise': 10000.0,
      'headingMatched': 9.0,
      'matchDistance': 17.2

In [13]:
data_waypoints = data["response"]["route"][0]["waypoint"]


List of route-matched GPS coordinates

In [14]:
matched_lat = [waypoint["mappedPosition"]["latitude"] for waypoint in data_waypoints]
matched_lng = [waypoint["mappedPosition"]["longitude"] for waypoint in data_waypoints]

In [15]:
df['matched_latitude'] = matched_lat
df['matched_longitude'] = matched_lng

# Comparison

In [16]:
BUBBLE_SIZE = 7
df['bubble_size'] = [BUBBLE_SIZE]*len(df)

In [17]:
CENTER = {"lat": 42.256, "lon": -85.64}

In [18]:
ZOOM = 15

In [19]:
fig = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    color_discrete_sequence=["black"],
    size="bubble_size",
    size_max=BUBBLE_SIZE,
    mapbox_style="open-street-map",
    center=CENTER,
    zoom=ZOOM,
)

fig.update_layout(margin={"l": 0, "r": 0, "t": 0, "b": 0})
fig.show()


In [20]:
fig = px.scatter_mapbox(
    df,
    lat="matched_latitude",
    lon="matched_longitude",
    color_discrete_sequence=["black"],
    size="bubble_size",
    size_max=BUBBLE_SIZE,
    mapbox_style="open-street-map",
    center=CENTER,
    zoom=ZOOM,
)

fig.update_layout(margin={"l": 0, "r": 0, "t": 0, "b": 0})
fig.show()